# 제주도 핫플레이스 - 인스타그램 크롤링

In [2]:
import requests
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [3]:
chromedirver = 'D:\download\chromedriver_win32\chromedriver'
driver = webdriver.Chrome(chromedirver)

In [ ]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [4]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [5]:
with open('password.txt') as f:
    password = f.read()

In [6]:
len(password)

10

In [7]:
with open('id.txt') as f:
    id = f.read()

In [8]:
len(id)

17

In [10]:
email = id
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')      #공백은 .으로 변경해줘야 함
input_email.clear()
input_email.send_keys(email)

In [13]:
input_pwd = driver.find_element_by_name('password')      #공백은 .으로 변경해줘야 함
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [17]:
# 브라우저 저장 나중에 하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass


In [18]:
# 알림설정 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

## 2.제주도맛집으로 검색

In [19]:
from urllib.parse import quote

In [23]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'

In [25]:
driver.get(url)
time.sleep(10)

In [26]:
driver.close()